In [ ]:
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font

folder = r'C:\Users\a\Desktop\etykiety'
xls_path = os.path.join(folder, 'etykiety.xls')
output_path = os.path.join(folder, 'etykiety_z_linkami.xlsx')

# 1. Wczytaj dane z pliku .xls (bez formatów)
df = pd.read_excel(xls_path, engine='xlrd')

# 2. Pobierz pliki PDF
pdf_files = sorted([f for f in os.listdir(folder) if f.lower().endswith('.pdf')])

# 3. Sprawdź, czy jest wystarczająco plików PDF
nazwy = df['Kod'].dropna().tolist()
if len(pdf_files) < len(nazwy):
    raise ValueError("Za mało plików PDF względem liczby nazw w kolumnie 'Kod'.")

# 4. Utwórz nowy skoroszyt Excel
wb = Workbook()
ws = wb.active
ws.title = "Etykiety"

# 5. Zapisz dane z DataFrame do Excela (bez formatów)
for r_idx, row in enumerate(df.itertuples(index=False), 1):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx+1, column=c_idx, value=value)  # +1, bo pierwszy wiersz na nagłówek

# 6. Zapisz nagłówki (kolumny)
for c_idx, col_name in enumerate(df.columns, 1):
    ws.cell(row=1, column=c_idx, value=col_name)

# 7. Znajdź numer kolumny 'Kod'
kod_col_idx = df.columns.get_loc('Kod') + 1  # +1 bo openpyxl liczy od 1

# 8. Dodaj linki do plików PDF w kolumnie 'Kod' (od wiersza 2)
for i, pdf in enumerate(pdf_files, start=2):
    cell = ws.cell(row=i, column=kod_col_idx)
    if cell.value:
        cell.hyperlink = pdf
        cell.font = Font(color="0000FF", underline="single")

# 9. Zapisz nowy plik
wb.save(output_path)
print(f"✔ Gotowe! Zapisano plik: {output_path}")
